In [19]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import random
#from selenium.webdriver.common.by import By
#from selenium.common.exceptions import TimeoutException
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.action_chains import ActionChains

In [2]:
seq = open('bpRNA_1m_90.fasta', 'r')
lines = seq.readlines()
data = {}
current_bprna_id = ""
for line in lines:
    line = line.replace('\r', '').replace('\n', '')
    if line[0] == ">":
        bprna_id = line[1:]
        current_bprna_id = bprna_id
    else:
        data[current_bprna_id] = line

In [3]:
small_range = 50
medium_range = 150
large_range = 500

In [4]:
def stem_energy(sp):
    se = 0
    if len(sp) > 1:
        for i in range(1, len(sp)):
            if sp[i] == "AU":
                if sp[i-1] == "AU": 
                    se += 0.9
                if sp[i-1] == "CG":
                    se += 2.2
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 1.1
                if sp[i-1] == "GU":
                    se += 0.6
                if sp[i-1] == "UG":
                    se += 1.4
            if sp[i] == "CG":
                if sp[i-1] == "AU": 
                    se += 2.1
                if sp[i-1] == "CG":
                    se += 3.3
                if sp[i-1] == "GC":
                    se += 2.4
                if sp[i-1] == "UA":
                    se += 2.1
                if sp[i-1] == "GU":
                    se += 1.4
                if sp[i-1] == "UG":
                    se += 2.1
            if sp[i] == "GC":
                if sp[i-1] == "AU": 
                    se += 2.4
                if sp[i-1] == "CG":
                    se += 3.4
                if sp[i-1] == "GC":
                    se += 3.3
                if sp[i-1] == "UA":
                    se += 2.2
                if sp[i-1] == "GU":
                    se += 1.5
                if sp[i-1] == "UG":
                    se += 2.5
            if sp[i] == "UA":
                if sp[i-1] == "AU": 
                    se += 1.3
                if sp[i-1] == "CG":
                    se += 2.4
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 0.9
                if sp[i-1] == "GU":
                    se += 1.0
                if sp[i-1] == "UG":
                    se += 1.3
            if sp[i] == "GU":
                if sp[i-1] == "AU": 
                    se += 1.3
                if sp[i-1] == "CG":
                    se += 2.5
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 1.4
                if sp[i-1] == "GU":
                    se += 0.5
                if sp[i-1] == "UG":
                    se += -1.3
            if sp[i] == "UG":
                if sp[i-1] == "AU": 
                    se += 1.0
                if sp[i-1] == "CG":
                    se += 1.5
                if sp[i-1] == "GC":
                    se += 1.4
                if sp[i-1] == "UA":
                    se += 0.6
                if sp[i-1] == "GU":
                    se += -0.3
                if sp[i-1] == "UG":
                    se += 0.5
    return se

In [53]:
def actual_stems(seq_ss, seq_ps):
    
    print(seq_ss)
    with open(seq_ss) as file:
        ss_lines = file.readlines()[3:]
    
    ps_lines = seq_ps
    rna = ps_lines
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    sp = []
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line
    
    for i in range(0, len(ss_lines)):
        line = ss_lines[i].strip().split()
        
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
            
        if (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
            
        if (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            temp.append(int(temp[1]-temp[0]-2*sl))
            temp.append(stem_energy(sp))
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
            sp = []
            
        if ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            temp.append(int(temp[1]-temp[0]-2*sl))
            temp.append(stem_energy(sp))
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 0
            sp = []
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
        
        last_line = line
        
    return stems_actual

In [6]:
def potential_pseudoknots(stems_potential, gamma):

    pseudoknots_potential = []

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
            #print(stem1)
            #print(stem2)
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i,j,1]
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
                
                pseudoknot[2] = gamma*np.log(stem1[2]*pseudoknot_sub_penalty(stem1[2])**2+stem2[2]*pseudoknot_sub_penalty(stem2[2])**2)
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [7]:
def pseudoknot_sub_penalty(length):
    return np.exp(0.572992*length+0.219677)

In [56]:
small_structures_pk = []
small_structures = []
medium_structures_pk = []
medium_structures = []
large_structures_pk = []
large_structures = []
counter = 0
for bprna_id, fasta in data.items():
    size_class = ""
    size = len(fasta)
    if (size>0 and size<=small_range):
        size_class = "small"
    elif (size>small_range and size<=medium_range):
        size_class = "medium"
    elif (size>medium_range and size<=large_range):
        size_class = "large"
    
    if size_class == "large":
        file = "./ctFiles/"+ bprna_id + ".ct"
        try:
            stems = actual_stems(file, fasta)
            if len(stems)>0:
                psudoknots = potential_pseudoknots(stems, 0.5)
                pk = False
                for pks in psudoknots:
                    if pks[2] != 1:
                        pk = True
                        break
                if size_class == "small" and pk:
                    small_structures_pk.append(bprna_id)
                elif size_class == "medium" and pk:
                    medium_structures_pk.append(bprna_id)
                elif size_class == "large" and pk:
                    large_structures_pk.append(bprna_id)
                elif size_class == "small" and not pk:
                    small_structures.append(bprna_id)
                elif size_class == "medium" and not pk:
                    medium_structures.append(bprna_id)
                elif size_class == "large" and not pk:
                    large_structures.append(bprna_id)
            
            counter+=1
            
        except:
            next

#print(small_structures_pk)
#print(medium_structures_pk[0])
#print(large_structures_pk[0])
#print(small_structures)
#print(medium_structures[0])
#print(large_structures[0])

./ctFiles/bpRNA_CRW_1236.ct
./ctFiles/bpRNA_CRW_1267.ct
./ctFiles/bpRNA_CRW_1328.ct
./ctFiles/bpRNA_CRW_1563.ct
./ctFiles/bpRNA_CRW_1619.ct
./ctFiles/bpRNA_CRW_1725.ct
./ctFiles/bpRNA_CRW_1824.ct
./ctFiles/bpRNA_CRW_2057.ct
./ctFiles/bpRNA_CRW_2852.ct
./ctFiles/bpRNA_CRW_2952.ct
./ctFiles/bpRNA_CRW_3718.ct
./ctFiles/bpRNA_CRW_3719.ct
./ctFiles/bpRNA_CRW_3731.ct
./ctFiles/bpRNA_CRW_3732.ct
./ctFiles/bpRNA_CRW_3736.ct
./ctFiles/bpRNA_CRW_3920.ct
./ctFiles/bpRNA_CRW_3922.ct
./ctFiles/bpRNA_CRW_3928.ct
./ctFiles/bpRNA_CRW_4004.ct
./ctFiles/bpRNA_CRW_4042.ct
./ctFiles/bpRNA_CRW_4052.ct
./ctFiles/bpRNA_CRW_4053.ct
./ctFiles/bpRNA_CRW_4063.ct
./ctFiles/bpRNA_CRW_4195.ct
./ctFiles/bpRNA_CRW_4197.ct
./ctFiles/bpRNA_CRW_4505.ct
./ctFiles/bpRNA_CRW_4754.ct
./ctFiles/bpRNA_CRW_4958.ct
./ctFiles/bpRNA_CRW_5019.ct
./ctFiles/bpRNA_CRW_5066.ct
./ctFiles/bpRNA_CRW_5094.ct
./ctFiles/bpRNA_CRW_5095.ct
./ctFiles/bpRNA_CRW_5394.ct
./ctFiles/bpRNA_CRW_6018.ct
./ctFiles/bpRNA_CRW_6403.ct
./ctFiles/bpRNA_CRW_

./ctFiles/bpRNA_tmRNA_606.ct
./ctFiles/bpRNA_tmRNA_607.ct
./ctFiles/bpRNA_tmRNA_608.ct
./ctFiles/bpRNA_tmRNA_610.ct
./ctFiles/bpRNA_tmRNA_611.ct
./ctFiles/bpRNA_tmRNA_612.ct
./ctFiles/bpRNA_tmRNA_613.ct
./ctFiles/bpRNA_tmRNA_614.ct
./ctFiles/bpRNA_tmRNA_615.ct
./ctFiles/bpRNA_tmRNA_619.ct
./ctFiles/bpRNA_tmRNA_620.ct
./ctFiles/bpRNA_tmRNA_621.ct
./ctFiles/bpRNA_tmRNA_622.ct
./ctFiles/bpRNA_tmRNA_624.ct
./ctFiles/bpRNA_tmRNA_640.ct
./ctFiles/bpRNA_tmRNA_643.ct
./ctFiles/bpRNA_tmRNA_646.ct
./ctFiles/bpRNA_tmRNA_648.ct
./ctFiles/bpRNA_tmRNA_650.ct
./ctFiles/bpRNA_tmRNA_651.ct
./ctFiles/bpRNA_tmRNA_661.ct
./ctFiles/bpRNA_tmRNA_662.ct
./ctFiles/bpRNA_tmRNA_664.ct
./ctFiles/bpRNA_tmRNA_666.ct
./ctFiles/bpRNA_tmRNA_667.ct
./ctFiles/bpRNA_tmRNA_668.ct
./ctFiles/bpRNA_tmRNA_669.ct
./ctFiles/bpRNA_tmRNA_670.ct
./ctFiles/bpRNA_tmRNA_671.ct
./ctFiles/bpRNA_tmRNA_672.ct
./ctFiles/bpRNA_tmRNA_674.ct
./ctFiles/bpRNA_tmRNA_676.ct
./ctFiles/bpRNA_tmRNA_677.ct
./ctFiles/bpRNA_tmRNA_678.ct
./ctFiles/bpRN

./ctFiles/bpRNA_RNP_350.ct
./ctFiles/bpRNA_RNP_351.ct
./ctFiles/bpRNA_RNP_352.ct
./ctFiles/bpRNA_RNP_353.ct
./ctFiles/bpRNA_RNP_355.ct
./ctFiles/bpRNA_RNP_357.ct
./ctFiles/bpRNA_RNP_358.ct
./ctFiles/bpRNA_RNP_360.ct
./ctFiles/bpRNA_RNP_362.ct
./ctFiles/bpRNA_RNP_365.ct
./ctFiles/bpRNA_RNP_368.ct
./ctFiles/bpRNA_RNP_374.ct
./ctFiles/bpRNA_RNP_375.ct
./ctFiles/bpRNA_RNP_377.ct
./ctFiles/bpRNA_RNP_378.ct
./ctFiles/bpRNA_RNP_379.ct
./ctFiles/bpRNA_RNP_380.ct
./ctFiles/bpRNA_RNP_381.ct
./ctFiles/bpRNA_RNP_382.ct
./ctFiles/bpRNA_RNP_383.ct
./ctFiles/bpRNA_RNP_388.ct
./ctFiles/bpRNA_RNP_394.ct
./ctFiles/bpRNA_RNP_398.ct
./ctFiles/bpRNA_RNP_400.ct
./ctFiles/bpRNA_RNP_405.ct
./ctFiles/bpRNA_RNP_407.ct
./ctFiles/bpRNA_RNP_408.ct
./ctFiles/bpRNA_RNP_409.ct
./ctFiles/bpRNA_RNP_411.ct
./ctFiles/bpRNA_RNP_412.ct
./ctFiles/bpRNA_RNP_413.ct
./ctFiles/bpRNA_RNP_414.ct
./ctFiles/bpRNA_RNP_416.ct
./ctFiles/bpRNA_RNP_419.ct
./ctFiles/bpRNA_RNP_420.ct
./ctFiles/bpRNA_RNP_421.ct
./ctFiles/bpRNA_RNP_423.ct
.

./ctFiles/bpRNA_RFAM_2348.ct
./ctFiles/bpRNA_RFAM_2349.ct
./ctFiles/bpRNA_RFAM_2350.ct
./ctFiles/bpRNA_RFAM_2351.ct
./ctFiles/bpRNA_RFAM_2364.ct
./ctFiles/bpRNA_RFAM_2370.ct
./ctFiles/bpRNA_RFAM_2371.ct
./ctFiles/bpRNA_RFAM_2372.ct
./ctFiles/bpRNA_RFAM_2374.ct
./ctFiles/bpRNA_RFAM_2375.ct
./ctFiles/bpRNA_RFAM_2377.ct
./ctFiles/bpRNA_RFAM_2378.ct
./ctFiles/bpRNA_RFAM_2379.ct
./ctFiles/bpRNA_RFAM_2380.ct
./ctFiles/bpRNA_RFAM_2381.ct
./ctFiles/bpRNA_RFAM_2382.ct
./ctFiles/bpRNA_RFAM_2383.ct
./ctFiles/bpRNA_RFAM_2384.ct
./ctFiles/bpRNA_RFAM_2385.ct
./ctFiles/bpRNA_RFAM_2386.ct
./ctFiles/bpRNA_RFAM_2387.ct
./ctFiles/bpRNA_RFAM_2388.ct
./ctFiles/bpRNA_RFAM_2389.ct
./ctFiles/bpRNA_RFAM_2390.ct
./ctFiles/bpRNA_RFAM_2391.ct
./ctFiles/bpRNA_RFAM_2392.ct
./ctFiles/bpRNA_RFAM_2393.ct
./ctFiles/bpRNA_RFAM_2394.ct
./ctFiles/bpRNA_RFAM_2395.ct
./ctFiles/bpRNA_RFAM_2396.ct
./ctFiles/bpRNA_RFAM_2397.ct
./ctFiles/bpRNA_RFAM_2398.ct
./ctFiles/bpRNA_RFAM_2399.ct
./ctFiles/bpRNA_RFAM_2400.ct
./ctFiles/bpRN

./ctFiles/bpRNA_RFAM_2785.ct
./ctFiles/bpRNA_RFAM_2787.ct
./ctFiles/bpRNA_RFAM_2788.ct
./ctFiles/bpRNA_RFAM_2789.ct
./ctFiles/bpRNA_RFAM_2790.ct
./ctFiles/bpRNA_RFAM_2791.ct
./ctFiles/bpRNA_RFAM_2793.ct
./ctFiles/bpRNA_RFAM_2794.ct
./ctFiles/bpRNA_RFAM_2796.ct
./ctFiles/bpRNA_RFAM_2827.ct
./ctFiles/bpRNA_RFAM_2828.ct
./ctFiles/bpRNA_RFAM_2829.ct
./ctFiles/bpRNA_RFAM_2830.ct
./ctFiles/bpRNA_RFAM_2831.ct
./ctFiles/bpRNA_RFAM_2832.ct
./ctFiles/bpRNA_RFAM_2833.ct
./ctFiles/bpRNA_RFAM_2834.ct
./ctFiles/bpRNA_RFAM_2835.ct
./ctFiles/bpRNA_RFAM_2836.ct
./ctFiles/bpRNA_RFAM_2837.ct
./ctFiles/bpRNA_RFAM_2840.ct
./ctFiles/bpRNA_RFAM_2845.ct
./ctFiles/bpRNA_RFAM_2853.ct
./ctFiles/bpRNA_RFAM_2854.ct
./ctFiles/bpRNA_RFAM_2855.ct
./ctFiles/bpRNA_RFAM_2859.ct
./ctFiles/bpRNA_RFAM_2864.ct
./ctFiles/bpRNA_RFAM_2871.ct
./ctFiles/bpRNA_RFAM_2876.ct
./ctFiles/bpRNA_RFAM_2877.ct
./ctFiles/bpRNA_RFAM_2878.ct
./ctFiles/bpRNA_RFAM_2882.ct
./ctFiles/bpRNA_RFAM_2886.ct
./ctFiles/bpRNA_RFAM_2889.ct
./ctFiles/bpRN

...

./ctFiles/bpRNA_RFAM_34098.ct
./ctFiles/bpRNA_RFAM_34111.ct
./ctFiles/bpRNA_RFAM_34112.ct
./ctFiles/bpRNA_RFAM_34122.ct
./ctFiles/bpRNA_RFAM_34136.ct
./ctFiles/bpRNA_RFAM_34144.ct
./ctFiles/bpRNA_RFAM_34152.ct
./ctFiles/bpRNA_RFAM_34153.ct
./ctFiles/bpRNA_RFAM_34154.ct
./ctFiles/bpRNA_RFAM_34156.ct
./ctFiles/bpRNA_RFAM_34183.ct
./ctFiles/bpRNA_RFAM_34185.ct
./ctFiles/bpRNA_RFAM_34188.ct
./ctFiles/bpRNA_RFAM_34189.ct
./ctFiles/bpRNA_RFAM_34190.ct
./ctFiles/bpRNA_RFAM_34192.ct
./ctFiles/bpRNA_RFAM_34196.ct
./ctFiles/bpRNA_RFAM_34197.ct
./ctFiles/bpRNA_RFAM_34198.ct
./ctFiles/bpRNA_RFAM_34200.ct
./ctFiles/bpRNA_RFAM_34205.ct
./ctFiles/bpRNA_RFAM_34206.ct
./ctFiles/bpRNA_RFAM_34207.ct
./ctFiles/bpRNA_RFAM_34210.ct
./ctFiles/bpRNA_RFAM_34212.ct
./ctFiles/bpRNA_RFAM_34214.ct
./ctFiles/bpRNA_RFAM_34219.ct
./ctFiles/bpRNA_RFAM_34466.ct
./ctFiles/bpRNA_RFAM_34522.ct
./ctFiles/bpRNA_RFAM_34523.ct
./ctFiles/bpRNA_RFAM_34525.ct
./ctFiles/bpRNA_RFAM_34527.ct
./ctFiles/bpRNA_RFAM_34528.ct
./ctFiles/

./ctFiles/bpRNA_RFAM_36280.ct
./ctFiles/bpRNA_RFAM_36289.ct
./ctFiles/bpRNA_RFAM_36290.ct
./ctFiles/bpRNA_RFAM_36297.ct
./ctFiles/bpRNA_RFAM_36305.ct
./ctFiles/bpRNA_RFAM_36308.ct
./ctFiles/bpRNA_RFAM_36356.ct
./ctFiles/bpRNA_RFAM_36357.ct
./ctFiles/bpRNA_RFAM_36369.ct
./ctFiles/bpRNA_RFAM_36370.ct
./ctFiles/bpRNA_RFAM_36380.ct
./ctFiles/bpRNA_RFAM_36382.ct
./ctFiles/bpRNA_RFAM_36384.ct
./ctFiles/bpRNA_RFAM_36385.ct
./ctFiles/bpRNA_RFAM_36386.ct
./ctFiles/bpRNA_RFAM_36388.ct
./ctFiles/bpRNA_RFAM_36390.ct
./ctFiles/bpRNA_RFAM_36391.ct
./ctFiles/bpRNA_RFAM_36392.ct
./ctFiles/bpRNA_RFAM_36393.ct
./ctFiles/bpRNA_RFAM_36394.ct
./ctFiles/bpRNA_RFAM_36395.ct
./ctFiles/bpRNA_RFAM_36396.ct
./ctFiles/bpRNA_RFAM_36508.ct
./ctFiles/bpRNA_RFAM_36509.ct
./ctFiles/bpRNA_RFAM_36510.ct
./ctFiles/bpRNA_RFAM_36512.ct
./ctFiles/bpRNA_RFAM_36513.ct
./ctFiles/bpRNA_RFAM_36514.ct
./ctFiles/bpRNA_RFAM_36515.ct
./ctFiles/bpRNA_RFAM_36519.ct
./ctFiles/bpRNA_RFAM_36520.ct
./ctFiles/bpRNA_RFAM_36524.ct
./ctFiles/

./ctFiles/bpRNA_RFAM_38176.ct
./ctFiles/bpRNA_RFAM_38177.ct
./ctFiles/bpRNA_RFAM_38180.ct
./ctFiles/bpRNA_RFAM_38182.ct
./ctFiles/bpRNA_RFAM_38186.ct
./ctFiles/bpRNA_RFAM_38188.ct
./ctFiles/bpRNA_RFAM_38189.ct
./ctFiles/bpRNA_RFAM_38190.ct
./ctFiles/bpRNA_RFAM_38191.ct
./ctFiles/bpRNA_RFAM_38192.ct
./ctFiles/bpRNA_RFAM_38194.ct
./ctFiles/bpRNA_RFAM_38196.ct
./ctFiles/bpRNA_RFAM_38198.ct
./ctFiles/bpRNA_RFAM_38199.ct
./ctFiles/bpRNA_RFAM_38200.ct
./ctFiles/bpRNA_RFAM_38201.ct
./ctFiles/bpRNA_RFAM_38204.ct
./ctFiles/bpRNA_RFAM_38205.ct
./ctFiles/bpRNA_RFAM_38206.ct
./ctFiles/bpRNA_RFAM_38207.ct
./ctFiles/bpRNA_RFAM_38208.ct
./ctFiles/bpRNA_RFAM_38211.ct
./ctFiles/bpRNA_RFAM_38212.ct
./ctFiles/bpRNA_RFAM_38213.ct
./ctFiles/bpRNA_RFAM_38216.ct
./ctFiles/bpRNA_RFAM_38218.ct
./ctFiles/bpRNA_RFAM_38219.ct
./ctFiles/bpRNA_RFAM_38221.ct
./ctFiles/bpRNA_RFAM_38224.ct
./ctFiles/bpRNA_RFAM_38225.ct
./ctFiles/bpRNA_RFAM_38226.ct
./ctFiles/bpRNA_RFAM_38230.ct
./ctFiles/bpRNA_RFAM_38231.ct
./ctFiles/

./ctFiles/bpRNA_RFAM_39207.ct
./ctFiles/bpRNA_RFAM_39214.ct
./ctFiles/bpRNA_RFAM_39218.ct
./ctFiles/bpRNA_RFAM_39225.ct
./ctFiles/bpRNA_RFAM_39227.ct
./ctFiles/bpRNA_RFAM_39229.ct
./ctFiles/bpRNA_RFAM_39250.ct
./ctFiles/bpRNA_RFAM_39251.ct
./ctFiles/bpRNA_RFAM_39289.ct
./ctFiles/bpRNA_RFAM_39306.ct
./ctFiles/bpRNA_RFAM_39310.ct
./ctFiles/bpRNA_RFAM_39315.ct
./ctFiles/bpRNA_RFAM_39319.ct
./ctFiles/bpRNA_RFAM_39694.ct
./ctFiles/bpRNA_RFAM_39753.ct
./ctFiles/bpRNA_RFAM_39754.ct
./ctFiles/bpRNA_RFAM_39767.ct
./ctFiles/bpRNA_RFAM_39770.ct
./ctFiles/bpRNA_RFAM_39771.ct
./ctFiles/bpRNA_RFAM_39772.ct
./ctFiles/bpRNA_RFAM_39773.ct
./ctFiles/bpRNA_RFAM_39774.ct
./ctFiles/bpRNA_RFAM_39777.ct
./ctFiles/bpRNA_RFAM_39778.ct
./ctFiles/bpRNA_RFAM_39779.ct
./ctFiles/bpRNA_RFAM_39781.ct
./ctFiles/bpRNA_RFAM_39782.ct
./ctFiles/bpRNA_RFAM_39815.ct
./ctFiles/bpRNA_RFAM_39816.ct
./ctFiles/bpRNA_RFAM_39817.ct
./ctFiles/bpRNA_RFAM_39818.ct
./ctFiles/bpRNA_RFAM_39819.ct
./ctFiles/bpRNA_RFAM_39821.ct
./ctFiles/

./ctFiles/bpRNA_RFAM_42193.ct
./ctFiles/bpRNA_RFAM_42194.ct
./ctFiles/bpRNA_RFAM_42195.ct
./ctFiles/bpRNA_RFAM_42196.ct
./ctFiles/bpRNA_RFAM_42203.ct
./ctFiles/bpRNA_RFAM_42218.ct
./ctFiles/bpRNA_RFAM_42219.ct
./ctFiles/bpRNA_RFAM_42220.ct
./ctFiles/bpRNA_RFAM_42221.ct
./ctFiles/bpRNA_RFAM_42222.ct
./ctFiles/bpRNA_RFAM_42223.ct
./ctFiles/bpRNA_RFAM_42224.ct
./ctFiles/bpRNA_RFAM_42225.ct
./ctFiles/bpRNA_RFAM_42227.ct
./ctFiles/bpRNA_RFAM_42301.ct
./ctFiles/bpRNA_RFAM_42303.ct
./ctFiles/bpRNA_RFAM_42304.ct
./ctFiles/bpRNA_RFAM_42305.ct
./ctFiles/bpRNA_RFAM_42306.ct
./ctFiles/bpRNA_RFAM_42307.ct
./ctFiles/bpRNA_RFAM_42308.ct
./ctFiles/bpRNA_RFAM_42309.ct
./ctFiles/bpRNA_RFAM_42310.ct
./ctFiles/bpRNA_RFAM_42312.ct
./ctFiles/bpRNA_RFAM_42313.ct
./ctFiles/bpRNA_RFAM_42314.ct
./ctFiles/bpRNA_RFAM_42316.ct
./ctFiles/bpRNA_RFAM_42317.ct
./ctFiles/bpRNA_RFAM_42319.ct
./ctFiles/bpRNA_RFAM_42320.ct
./ctFiles/bpRNA_RFAM_42342.ct
./ctFiles/bpRNA_RFAM_42348.ct
./ctFiles/bpRNA_RFAM_42435.ct
./ctFiles/

In [57]:
print(large_structures_pk)

['bpRNA_CRW_1328', 'bpRNA_CRW_1563', 'bpRNA_CRW_1619', 'bpRNA_CRW_1725', 'bpRNA_CRW_1824', 'bpRNA_CRW_3719', 'bpRNA_CRW_3732', 'bpRNA_CRW_3920', 'bpRNA_CRW_4004', 'bpRNA_CRW_4754', 'bpRNA_CRW_4958', 'bpRNA_CRW_5066', 'bpRNA_CRW_5394', 'bpRNA_CRW_6018', 'bpRNA_CRW_6403', 'bpRNA_CRW_6618', 'bpRNA_CRW_6826', 'bpRNA_CRW_6836', 'bpRNA_CRW_7455', 'bpRNA_CRW_7488', 'bpRNA_CRW_7494', 'bpRNA_CRW_7501', 'bpRNA_CRW_7502', 'bpRNA_CRW_7614', 'bpRNA_CRW_8054', 'bpRNA_CRW_8187', 'bpRNA_CRW_8656', 'bpRNA_CRW_8657', 'bpRNA_CRW_9265', 'bpRNA_CRW_9390', 'bpRNA_CRW_9431', 'bpRNA_CRW_9458', 'bpRNA_CRW_9473', 'bpRNA_CRW_9756', 'bpRNA_CRW_10181', 'bpRNA_CRW_10337', 'bpRNA_CRW_11137', 'bpRNA_CRW_16959', 'bpRNA_CRW_17477', 'bpRNA_CRW_20554', 'bpRNA_CRW_54544', 'bpRNA_CRW_54545', 'bpRNA_CRW_54546', 'bpRNA_CRW_54547', 'bpRNA_CRW_54549', 'bpRNA_CRW_54550', 'bpRNA_CRW_54553', 'bpRNA_CRW_54561', 'bpRNA_CRW_54562', 'bpRNA_CRW_54564', 'bpRNA_CRW_54567', 'bpRNA_CRW_54568', 'bpRNA_CRW_54569', 'bpRNA_CRW_54570', 'bpRNA_

In [60]:
pks = "/woutPks"
size = "/l/"
id_list = random.sample(large_structures,35)

In [61]:
url = "http://bprna.cgrb.oregonstate.edu/"
for i in id_list:
    fasta = url+"fastaFiles/"+i+".fasta"
    ct = url+"ctFiles/"+i+".ct"
    browser = webdriver.Chrome(executable_path="C:/Users/juan-/OneDrive/Escritorio/Repos/RNA-Folding/chromedriver")
    browser.get(fasta)
    fasta_text = browser.find_elements_by_css_selector('pre')[0]
    with open("./dataset"+pks+size+i+".fasta.txt", 'w') as f:
        f.write(fasta_text.text)
    browser = webdriver.Chrome(executable_path="C:/Users/juan-/OneDrive/Escritorio/Repos/RNA-Folding/chromedriver")
    browser.get(ct)
    ct_text = browser.find_elements_by_css_selector('pre')[0]
    with open("./dataset"+pks+size+i+".ct.txt", 'w') as f:
        f.write(ct_text.text)
 

C:\Users\juan-\AppData\Local\Temp/ipykernel_1452/2141046670.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:/Users/juan-/OneDrive/Escritorio/Repos/RNA-Folding/chromedriver")
C:\Users\juan-\AppData\Local\Temp/ipykernel_1452/2141046670.py:7: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fasta_text = browser.find_elements_by_css_selector('pre')[0]
C:\Users\juan-\AppData\Local\Temp/ipykernel_1452/2141046670.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:/Users/juan-/OneDrive/Escritorio/Repos/RNA-Folding/chromedriver")
C:\Users\juan-\AppData\Local\Temp/ipykernel_1452/2141046670.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selecto